In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import re, string, unicodedata
import os
import inflect
import numpy as np
import pandas as pd
import collections
import networkx as nx
import time
import pickle

import nltk
from nltk import word_tokenize, sent_tokenize
from nltk import download
from nltk.corpus import stopwords


from sklearn.metrics.pairwise import cosine_similarity


%pip install stanza
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=d25b52bfb785c61c1e763cb28c62abcc48a68e9defae685ff8c21f422be3a394
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
download('stopwords')
download('punkt')
stanza.download('es')
nlp = stanza.Pipeline('es')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2022-01-12 04:46:19 INFO: Downloading default packages for language: es (Spanish)...


2022-01-12 04:46:37 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-01-12 04:46:37 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-01-12 04:46:37 INFO: Use device: cpu
2022-01-12 04:46:37 INFO: Loading: tokenize
2022-01-12 04:46:37 INFO: Loading: mwt
2022-01-12 04:46:37 INFO: Loading: pos
2022-01-12 04:46:38 INFO: Loading: lemma
2022-01-12 04:46:38 INFO: Loading: depparse
2022-01-12 04:46:39 INFO: Loading: ner
2022-01-12 04:46:41 INFO: Done loading processors!


In [ ]:
#Convierto cada tweet del dataset en un vector de palabras
from nltk import word_tokenize
def ConvertirDataAnterior(corpusBase):
  textos = []
  for y in [corpusBase[i][0] for i in range(len(corpusBase))]:
      doc = word_tokenize(y)
      textos.append(doc)
  return textos

def ConvertirData(data):
  comments = []
  for line in data:
      comments.append(str(line).split(' '))
  return comments

In [ ]:
DataFinal = pd.read_excel('/content/gdrive/My Drive/ART_EMERGENCY/GLOVE_CODIGO/4DataFinalLemaFiltro.xlsx')

FUNCIONES

In [ ]:
def Knn(Matriz,tipo, k):
      labels = tipo
      pred_label = []
      etiquetar = []
      indice = 0
      for x in labels:
          if(indice != len(labels)-1):
            if(Matriz[indice]<=0.25): #Filtro que asigan directament 0 cuando el valor de comparacion sea cero o menor a cero
              etiquetar.append((Matriz[indice], 0))
            else:
              etiquetar.append((Matriz[indice], x))
          indice += 1
      etiquetar.sort(reverse=True)
      neighbors = etiquetar[:k]
      votes = []
      for neighbor in neighbors:
                votes.append(neighbor[1])
      counter = collections.Counter(votes) #Determina el que mas se repite
      pred_label.append(counter.most_common()[0][0])
      return pred_label[0]

MODELO GLOVE

In [ ]:
! pip install glove-python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 15.9 MB/s eta 0:00:00


In [ ]:
    import numpy as np
    from sklearn.model_selection import train_test_split
    import pandas as pd
    from glove import Corpus, Glove
    from sklearn.metrics import accuracy_score

      
    def fit_Glove(textos):
      corpus = Corpus()
      corpus.fit(textos, window=10)

      # entrenar el modelo GloVe con el corpus
      glove = Glove(no_components=100, learning_rate=0.05)
      glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
      glove.add_dictionary(corpus.dictionary)

      return glove

    def modeloGlove(Tweet,glove_model, data_vector):
      #Tweet = tweet
      if len(Tweet.split()) <= 2: #Se valida que el tweet preprocesado tenga un minino de palabras
        Tweet = 'falso'  #Se le asigna el texto 'vacio' el cual sera detectado como no emergencia
      
      tweet_vector = np.mean([glove_model.word_vectors[glove_model.dictionary[word]] for word in Tweet.split() if word in glove_model.dictionary], axis=0)
      
      sim = []

      for line in data_vector:
        similarity = (line @ tweet_vector) / (np.linalg.norm (line) * np.linalg.norm (tweet_vector)) 
        sim.append(similarity)

      return sim


    def predict_Glove(Train, TrainOri, y_train, tweets):
      prediccion = []
      glove_model = fit_Glove(Train)
      
      data_vector = []
      for comment in TrainOri:
        words_vectors = np.mean([glove_model.word_vectors[glove_model.dictionary[word]] for word in str(comment).split() if word in glove_model.dictionary],axis=0)
        # Aqui puedes promediar los vectores de las palabras o utilizar otra técnica para representar al comentario
        data_vector.append(words_vectors)

      for tweet in [tweets[i][0] for i in range(len(tweets))]:
        Similaridad= modeloGlove(tweet,glove_model,data_vector)
        prediccion.append(k_predict_Glove(Similaridad,y_train))
      return prediccion

    def k_predict_Glove(Similaridad, y_train):
      k_1 = Knn(Similaridad,y_train, 1)
      k_3 = Knn(Similaridad,y_train, 3)
      k_5 = Knn(Similaridad,y_train, 5)
      k_7 = Knn(Similaridad,y_train, 7)
      k_9 = Knn(Similaridad,y_train, 9)
      k_11 = Knn(Similaridad,y_train, 11)
      return k_1, k_3, k_5, k_7, k_9, k_11 


In [ ]:
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X = DataFinal['text']
y = DataFinal['tipo']

kf = KFold(n_splits=8, shuffle=True, random_state=2) 

In [ ]:
#MODELO GLOVE 

In [ ]:

from sklearn.metrics import confusion_matrix
nombre = []
accuracy = []
precision = []
recall = []
sensitivy = []
f1 = []
log = []

indice_fold=1



for rep in range(30):
  indice_fold=1
  for entrenamiento_indice, prueba_indice in kf.split(X):
    #print(entrenamiento_indice,prueba_indice)
    X_train, X_test = X[entrenamiento_indice],X[prueba_indice]
    y_train, y_test = y[entrenamiento_indice],y[prueba_indice]
    X_train = X_train.values.reshape([X_train.values.shape[0],1])
    X_test = X_test.values.reshape([X_test.values.shape[0],1]) 
    
    X_proce = ConvertirData(X[entrenamiento_indice]) 
    y_pred =  predict_Glove(X_proce,X[entrenamiento_indice],y_train,X_test)
    y_pred = np.array(y_pred)
    a = [1,3,5,7,9,11]
    print(' COMPLETADO:')
    for i in range(6):
      name = 'GLOVE K'+str(a[i])+', FOLD '+str(indice_fold)+', REPETICION '+str(rep +1)
      nombre.append(name)
      tn, fp, fn, tp = confusion_matrix(y_test, y_pred[:,i]).ravel()
      accuracy.append(metrics.accuracy_score(y_test,y_pred[:,i]))
      precision.append(metrics.precision_score(y_test,y_pred[:,i]))
      f1.append(metrics.f1_score(y_test,y_pred[:,i]))
      log.append(metrics.log_loss(y_test,y_pred[:,i]))
      recall.append( tn / float(tn + fp))
      sensitivy.append(tp / float(tp + fn))
      print(' '+name)
      #print(accuracy, precision, recall, sensitivy)
    indice_fold = indice_fold + 1 

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
 COMPLETADO:
 GLOVE K1, FOLD 5, REPETICION 14
 GLOVE K3, FOLD 5, REPETICION 14
 GLOVE K5, FOLD 5, REPETICION 14
 GLOVE K7, FOLD 5, REPETICION 14
 GLOVE K9, FOLD 5, REPETICION 14
 GLOVE K11, FOLD 5, REPETICION 14
Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
 COMPLETADO:
 GLOVE K1, FOLD 6, REPETICION 14
 GLOVE K3, FOLD 6, REPETICION 14
 GLOVE K5, FOLD 6, REPETICION 14
 GLOVE K7, FOLD 6, REPETICION 14
 GLOVE K9, FOLD 6, REPETICION 14
 GLOVE K11, FOLD 6, REPETICION 14
Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 

In [ ]:
resultado2=[nombre,accuracy,precision,recall,sensitivy,f1,log]
Final2 = pd.DataFrame(resultado2)
Final2.T

,0,1,2,3,4,5,6
0,"GLOVE K1, FOLD 1, REPETICION 1",0.799401,0.773973,0.823529,0.768707,0.771331,6.928516
1,"GLOVE K3, FOLD 1, REPETICION 1",0.826347,0.815603,0.860963,0.782313,0.798611,5.997814
2,"GLOVE K5, FOLD 1, REPETICION 1",0.838323,0.839416,0.882353,0.782313,0.809859,5.584166
3,"GLOVE K7, FOLD 1, REPETICION 1",0.859281,0.890625,0.925134,0.77551,0.829091,4.86028
4,"GLOVE K9, FOLD 1, REPETICION 1",0.853293,0.876923,0.914439,0.77551,0.823105,5.067104
...,...,...,...,...,...,...,...
1435,"GLOVE K3, FOLD 8, REPETICION 30",0.810811,0.810458,0.834286,0.78481,0.797428,6.534433
1436,"GLOVE K5, FOLD 8, REPETICION 30",0.810811,0.810458,0.834286,0.78481,0.797428,6.534433
1437,"GLOVE K7, FOLD 8, REPETICION 30",0.822823,0.836735,0.862857,0.778481,0.806557,6.119541
1438,"GLOVE K9, FOLD 8, REPETICION 30",0.813814,0.838028,0.868571,0.753165,0.793333,6.430698


In [ ]:
Final2.T.to_excel('Pruebas_Glove.xlsx', index=False)
!cp Pruebas_Glove.xlsx '/content/gdrive/My Drive/ART_EMERGENCY/PRUEBAS/'